## Imports

In [1]:
import lxml.html as html
import math
import csv
import time
import requests

## Configurations

In [2]:
# Trustpilot review page
basePage = 'http://www.trustpilot.com/review/'
reviewSite = 'www.skype.com'
reviewPage = basePage + reviewSite

# Data file to save to
datafile = 'dataSkype.csv'

# Trustpilot default 
resultsPerPage = 20 

print('Scraper set for ' + reviewPage + ' - saving result to ' + datafile)

Scraper set for http://www.trustpilot.com/review/www.skype.com - saving result to dataSkype.csv


In [8]:
## Count amount of pages to scrape

# Get page, skipping HTTPS as it gives certificate errors
page = requests.get(reviewPage, verify=False)
tree = html.fromstring(page.content)


# Total amount of ratings
ratingCount = tree.xpath('//span[@class="headline__review-count"]')
ratingCount = int(ratingCount[0].text.replace(',',''))

# Amount of chunks to consider for displaying processing output 
# For ex. 10 means output progress for every 10th of the data
tot_chunks = 20

# Throttling to avoid spamming page with requests
# With sleepTime seconds between every page request
throttle = False
sleepTime = 2

# Total pages to scrape
pages = math.ceil(ratingCount / resultsPerPage)
print('Found total of ' + str(pages) + ' pages to scrape')

[<Element span at 0x194198fbae8>]
Found total of 52 pages to scrape


C:\Users\dany_\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [27]:
## Main scraping section

with open(datafile, 'w', newline='', encoding='utf8') as csvfile:
    
    # Tab delimited to allow for special characters
    datawriter = csv.writer(csvfile, delimiter='\t')
    print('Processing..')
    for i in range(1,pages+1):
        
        # Sleep if throttle enabled
        if(throttle): time.sleep(sleepTime)

        page = requests.get(reviewPage + '?page=' + str(i))
        tree = html.fromstring(page.content)
        
        # Each item below scrapes a pages review titles, bodies, ratings and languages. 
        titles = tree.xpath('//a[@class="link link--large link--dark"]')
        bodies = tree.xpath('//p[@class="review-content__text"]')
        ratings = tree.xpath('//div[@data-status]')
        langs = tree.xpath("//h3[starts-with(@class, 'review-title')]")
        
        for idx,e in enumerate(bodies):
            
            # Progress counting, outputs for every processed chunk
            reviewNumber = (idx + 1) + 20*(i-1)
            chunk = int(ratingCount / tot_chunks)
            if(reviewNumber % chunk == 0): 
                print('Processed ' + str(reviewNumber) + '/'  + str(ratingCount) + ' ratings')
            
            # Title of comment
            title = titles[idx].text_content()
            
            # Body of comment
            body = e.text_content().strip()
            
            # The rating is the 5th from last element
            rating = 3 #ratings[idx].get('data-status').split(' ')[-5] 
            
            # Language is 2nd element of h3 tag
            lang = "en" # langs[idx].get('class').split(' ')[1]
            
            datawriter.writerow([title,body,rating,lang])
    print('Processed ' + str(ratingCount) + '/' + str(ratingCount) + ' ratings.. Finished!')

Processing..
Processed 51/1028 ratings
Processed 102/1028 ratings
Processed 153/1028 ratings
Processed 204/1028 ratings
Processed 255/1028 ratings
Processed 306/1028 ratings
Processed 357/1028 ratings
Processed 408/1028 ratings
Processed 459/1028 ratings
Processed 510/1028 ratings
Processed 561/1028 ratings
Processed 612/1028 ratings
Processed 663/1028 ratings
Processed 714/1028 ratings
Processed 765/1028 ratings
Processed 816/1028 ratings
Processed 867/1028 ratings
Processed 918/1028 ratings
Processed 969/1028 ratings
Processed 1020/1028 ratings
Processed 1028/1028 ratings.. Finished!
